<a href="https://colab.research.google.com/github/sanathdavis/python_test/blob/main/Gokul_Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def get_urls():
  make = 'chrysler'
  models = [
      "200",
      "300",
      "300c",
      "aspen",
      "aspen_hybrid",
      "cirrus",
      "concorde",
      "crossfire",
      "grand_voyager",
      "imperial",
      "lebaron",
      "lhs"
      "new_yorker",
      "pacifica",
      "pacifica_hybrid",
      "prowler",
      "pt_cruiser",
      "sebring",
      "town_and_country",
      "voyager"
  ]

  years = [
      "2014",
      "2015",
      "2016",
      "2017",
      "2018",
      "2019",
      "2020",
      "2021",
      "2022"
  ]

  urls = []
  for model in models:
    for year in years:
      urls.append({
          'url':'https://www.cars.com/research/chrysler-' + model + '-' + year + '/consumer-reviews/?page_size=200',          
          'make' : make,
          'model' : model,
          'year' : year
          })
      
  #3 cars have more than 200 reviewes. So adding their second pages to the list of urls
  urls.append({
      'url' : 'https://www.cars.com/research/chrysler-200-2015/consumer-reviews/?page=2&page_size=200',
      'make' : make,
      'model' : model,
      'year' : year
  })

  urls.append({
      'url' : 'https://www.cars.com/research/chrysler-pacifica-2017/consumer-reviews/?page=2&page_size=200',
      'make' : make,
      'model' : model,
      'year' : year
  })

  urls.append({
      'url' : 'https://www.cars.com/research/chrysler-pacifica-2018/consumer-reviews/?page=2&page_size=200',
      'make' : make,
      'model' : model,
      'year' : year
  })


  #test
  # urls = [{
  #     'url' : 'https://www.cars.com/research/chrysler-concorde-2014/consumer-reviews/',
  #     'make' : 'make',
  #     'model' : 'model',
  #     'year' : 'year'
  # }]

  return urls

In [ ]:
def get_soup_from_url(url):
  url_response = requests.get(url, allow_redirects=False)
  return BeautifulSoup(url_response.text, 'html.parser')

In [ ]:
# #main rating value
# main_rating_block = soup.find('div', {'class': 'sds-rating--big'})
# rating = float(main_rating_block.span.string)
# print(rating)

In [ ]:
# #recommended by percentage value

# recommended_by_text = soup.find(text=re.compile('of drivers recommend this car'))
# recommended_by_percent = float(recommended_by_text.split('%')[0])
# print(recommended_by_percent)


In [ ]:
#number of reviews
def get_number_of_reviews(soup):
  number_of_reviews_html = soup.find('a', {'class': 'sds-rating__link sds-button-link'})
  try:
    number_of_reviews = number_of_reviews_html.string.split(" ")[0].replace('(','')
    return int(number_of_reviews)
  except:
    #print(number_of_reviews_html, soup.title)
    return 0
  
  

In [ ]:
# #total review breakdown
# review_type_html = soup.find('span', {'class': 'sds-definition-list__value'})
# review_type = review_type_html.string
# review_value = review_type_html.next_element
# print(review_type, review_value)

In [ ]:
def get_reviews_from_soup(soup, url, make, model, year):
  reviews = []
  reviews_block = soup.findAll('div', {'class': 'consumer-review-container'})
  for review in reviews_block:
    user_total_review = float(review.find("span", {"class": "sds-rating__count"}).string) 

    user_review_title = review.find("h3", {"class": "sds-heading--7"}).string

    review_breakdown_block = review.findAll("span", {"class": "sds-definition-list__value"})
    review_breakdown =  ['N/A' if '—' in i.string else float(i.string) for i in review_breakdown_block]      

    reviewed_on = review.find("div", {"class": "review-byline review-section"}).div.string
    reviewed_by = review.find("div", {"class": "review-byline review-section"}).find_all('div')[1].string
    review_type = review.find("div", {"class": "review-type"}).string

    review_reasons_block = review.find("ul", {"class": "review-reasons"}).find_all('li')
    purchased_new_car = 0 if 'Used' in review_reasons_block[0].get_text() else 1
    if len(review_reasons_block) >= 3:      
      used_for = review_reasons_block[1].get_text().replace('Used for', '')
      recommend_or_not = 0 if 'not' in review_reasons_block[2].get_text() else 1
    else:
      used_for = 'N/A'
      recommend_or_not = 0 if 'not' in review_reasons_block[1].get_text() else 1

    review_found_useful_by = int(review.find("div", {"class": "consumer-rating-counts"}).get_text().split(' ')[0])

    review_body = review.find("div", {"class": "review-byline review-section"}).find_next_sibling("div").p.string

    reviews.append({
        'make' : make,
        'model' : model,
        'year' : year,
        'user_total_review' : user_total_review,
        'comfort' : review_breakdown[0],
        'interior_design' : review_breakdown[1],
        'performance' : review_breakdown[2],
        'value_for_the_money' : review_breakdown[3],
        'exterior_styling' : review_breakdown[4],
        'reliability' : review_breakdown[5],
        'user_review_title': user_review_title,
        'review_body':review_body,
        'reviewed_on': reviewed_on,
        'reviewed_by' : reviewed_by,
        'review_type' : review_type,
        'purchased_new_car' : purchased_new_car,
        'used_for' : used_for,
        'recommend_or_not' : recommend_or_not,
        'review_found_useful_by' : review_found_useful_by,
        'url' : url
    })

  return reviews

In [ ]:

def download_csv(all_reviews):
  column_headings = [
      'Make', 
      'Model',
      'Year', 
      'Total Review', 
      'Comfort', 
      'Interior design', 
      'Performance', 
      'Value for the money', 
      'Exterior styling', 
      'Reliability', 
      'Review Title', 
      'Review Body', 
      'Reviewd On', 
      'Reviewed By', 
      'Review Type', 
      'Purchased New Car', 
      'Used For', 
      'Recomend', 
      'Review Found Useful By',
      'Url'
      ]

  with open('car_reviews.csv', mode='w') as employee_file:
      employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
      employee_writer.writerow(column_headings)
      for review in all_reviews:
        employee_writer.writerow(review.values())


In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import csv

def main() :
  all_reviews = []
  urls = get_urls()
  for url in urls:
    soup = get_soup_from_url(url['url'])
    if (soup.get_text() != 'You are being redirected.'):
      number_of_reviews = get_number_of_reviews(soup)
      if (number_of_reviews > 0):  
        if (number_of_reviews > 200):
          print("More than 200 reviews in " + url['url'])
        all_reviews = all_reviews + get_reviews_from_soup(soup, url['url'], url['make'], url['model'], url['year'])
        download_csv(all_reviews)
      else:
        print("No reviews in " + url['url'])        
    else:
      print('Url does not exist ' + url['url'])

main()

More than 200 reviews in https://www.cars.com/research/chrysler-200-2015/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-200-2018/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-200-2019/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-200-2020/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-200-2021/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-200-2022/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-300c-2019/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-300c-2020/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-300c-2021/consumer-reviews/?page_size=200
Url does not exist https://www.cars.com/research/chrysler-300c-2022/consumer-reviews/?page_size=200
